In [32]:
import nibabel as nib
import os 
from scipy import ndimage
import numpy as np


def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load("\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images")
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [33]:
# Folder "CT-0" consist of CT scans having normal lung tissue,
# no CT-signs of viral pneumonia.
normal_scan_paths = [
    os.path.join(os.getcwd(), "\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images", x)
    for x in os.listdir("\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images")
]
# Folder "CT-23" consist of CT scans having several ground-glass opacifications,
# involvement of lung parenchyma.
abnormal_scan_paths = [
    os.path.join(os.getcwd(), "\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images", x)
    for x in os.listdir("\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images")
]

print("CT scans with normal lung tissue: " + str(len(normal_scan_paths)))
print("CT scans with abnormal lung tissue: " + str(len(abnormal_scan_paths)))

CT scans with normal lung tissue: 24
CT scans with abnormal lung tissue: 24


In [34]:
normal_scan_paths

['c:\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images\\ct_1054_image.nii.gz',
 'c:\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images\\ct_1112_image.nii.gz',
 'c:\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images\\ct_1120_image.nii.gz',
 'c:\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images\\ct_1124_image.nii.gz',
 'c:\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images\\ct_1126_image.nii.gz',
 'c:\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images\\ct_1127_image.nii.gz',
 'c:\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images\\ct_1128_image.nii.gz',
 'c:\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images\\ct_1129_image.nii.gz',
 'c:\\Users\\HPC\\Desktop\\minor_files\\ImageCHD_dataset\\images_with_label\\images\\ct_1133_image.nii.gz',
 'c:\\Users\\HPC\\Desktop\\m

In [35]:
# Read and process the scans.
# Each scan is resized across height, width, and depth and rescaled.
abnormal_scans = np.array([process_scan(path) for path in abnormal_scan_paths])
normal_scans = np.array([process_scan(path) for path in normal_scan_paths])

# For the CT scans having presence of viral pneumonia
# assign 1, for the normal ones assign 0.
abnormal_labels = np.array([1 for _ in range(len(abnormal_scans))])
normal_labels = np.array([0 for _ in range(len(normal_scans))])

# Split data in the ratio 70-30 for training and validation.
x_train = np.concatenate((abnormal_scans[:10], normal_scans[:10]), axis=0)
y_train = np.concatenate((abnormal_labels[:10], normal_labels[:10]), axis=0)
x_val = np.concatenate((abnormal_scans[10:], normal_scans[10:]), axis=0)
y_val = np.concatenate((abnormal_labels[10:], normal_labels[10:]), axis=0)
print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_val.shape[0])
)

ImageFileError: Cannot work out file type of "\Users\HPC\Desktop\minor_files\ImageCHD_dataset\images_with_label\images"